# Project 1, Part 3, Executive Questions: Product Specific Questions

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: John (Jack) Galvin

Year: 2022

Semester: Spring

Section: 9


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
# Function to run a select query and return rows in a pandas dataframe
# Pandas puts all numeric values from postgres to float
# If it will fit in an integer, change it to integer


def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
# Connect to Postgres

connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
# Create a cursor for the connection

cursor = connection.cursor()

# Instructions for Specific Questions

For each specific quesion, you must write 1 and only 1 query that answers that specific question.  The results of your query must be displayed in a Pandas dataframe as demonstrated in the labs

All output should be sorted.

For store name, use the city name the store is located in.

When sorting by store name, sort in alphabetical order.

For meal name, use the product description.

When sorting by meal name, sort in alphabetical order.

For month, use the full month name: January, February, March, etc.

When sorting by month, sort in calendar order: January, February, March, etc.  You may find it useful to include a month number to help with the sort.  It's ok to leave it in the output.

For day of week, use the full day of week name: Sunday, Monday, Tuesday, etc.

When sorting by day of week, sort in day of week order: Sunday, Monday, Tuesday, etc. You may find it usefule to include a day of week number to help with the sort.  It's ok to leave it in the output.


# 1.3.1 How many meals were purchased for all of AGM?

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select sum(quantity) as total_meals
from line_items

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,total_meals
0,8228284


# 1.3.2 How many meals were purchased for all of AGM by meal?

Sort by meal name in alphabetical order

In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select p.description as product,
       sum(l.quantity) as total_quantity_sold
from line_items as l
     join products as p
         on l.product_id = p.product_id
group by product
order by product

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,product,total_quantity_sold
0,Brocolli Stir Fry,913984
1,Chicken Salad,228561
2,Curry Chicken,1368884
3,Eggplant Lasagna,1599058
4,Pistachio Salmon,1828778
5,Spinach Orzo,456769
6,Teriyaki Chicken,1145013
7,Tilapia Piccata,687237


# 1.3.3 How many meals were purchased by store and by meal?

Sort by store name in alphabetical order, then by meal name in alphabetical order

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select s.city as store,
        p.description as product,
        sum(l.quantity) as total_quantity_sold,
        s.city as store
from stores as s
    join sales as sa
        on s.store_id = sa.store_id
    join line_items as l
        on sa.store_id = l.store_id and sa.sale_id = l.sale_id
    join products as p
        on l.product_id = p.product_id
group by store, product
order by store, product

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,product,total_quantity_sold,store
0,Brocolli Stir Fry,232038,Berkeley
1,Chicken Salad,57719,Berkeley
2,Curry Chicken,346508,Berkeley
3,Eggplant Lasagna,405637,Berkeley
4,Pistachio Salmon,464274,Berkeley
5,Spinach Orzo,115469,Berkeley
6,Teriyaki Chicken,290858,Berkeley
7,Tilapia Piccata,174252,Berkeley
8,Brocolli Stir Fry,179885,Dallas
9,Chicken Salad,44756,Dallas


# 1.3.4 How many meals were purchased by month?

Sort by month in calendar order

# 1.3.5 How many meals were purchased by month and meal?

Sort by month in calendar order, then by meal name in alphabetical order

# 1.3.6 How many meals were purchased by day of week and meal?

Sort by day of week in day of week order, then by meal name in alphabetical order

# 1.3.7 Average number of meals per sale for all of AGM


# 1.3.8 Average number of meals per sale by store

Sort by store name in alphabetical order


# 1.3.9 The Executives have also asked you to provide your best example of a data visualization for one of the above queries

You may choose any one of the above queries. 

It's best to select the one that you think will provide the most interesting data visualization.  

You are allowed to add a pivot as demonstrated in the labs if it is needed for your data visualization. 